What is the best economic domain/industry to invest in (return, risk, stability)? Over all countries, over specific countries/world regions

How is best defined?
Return, risk, stability

Examine a major industries (food, financial, health etc) and determine a goodness score consisting of return, stability and risk

Examine these industries in general and divided by regions etc
(If you are in Africa it is best to invest here etc)

Focus on s&p 500
1. analyse different industries in s&p 500

In [ ]:
print("hello world")